<a href="https://colab.research.google.com/github/Mike030668/MIPT_magistratura/blob/main/NLP_generation/Generation/notebooks/Telegram_tyni_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *Устанавливаем необходимый пакет*

Для взаимодействия с Telegram API средствами языка Python разработаны несколько модулей. Самый простой из них — Telebot.


# variant load model with peft

In [ ]:
!git clone https://github.com/Mike030668/MIPT_magistratura.git -q
!pip3 install pytelegrambotapi -q
!pip install trl transformers accelerate git+https://github.com/huggingface/peft.git -Uqqq
!pip install bitsandbytes einops wandb -Uqqq

import gdown  # библиотека по работе с файлами в том числе и с гугл_диска
import pandas as pd
import numpy as np
import random
import torch
import gc
from transformers import AutoTokenizer, AutoModel
import sys

from peft import get_peft_model, PeftConfig, PeftModel, LoraConfig, prepare_model_for_kbit_training
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, GenerationConfig

import warnings
warnings.filterwarnings('ignore')


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MAIN_DIR =  "/content/MIPT_magistratura/NLP_generation/Generation"

sys.path.append(MAIN_DIR)
from utils.download import gd_folder_download
from utils.constant import SHARE_DIR, TOKEN
from utils.generate import generate_answer, get_prompt, ROLES


# Подгружаем с публичной папки по https
dir_to = MAIN_DIR + "/TinyLamma/"

# код потребует гугл авторизацию любую из имеющихся
model_dir = gd_folder_download(SHARE_DIR, dir_to)

config = PeftConfig.from_pretrained(model_dir)
# Loading the model with double quantization
model_name = "PY007/TinyLlama-1.1B-step-50K-105b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)


model = AutoModelForCausalLM.from_pretrained(
	config.base_model_name_or_path,
	quantization_config=bnb_config,
	trust_remote_code=True,
	device_map='auto'

	)

# Add additional trainable layers to a PEFT adapter
# https://huggingface.co/docs/transformers/main/peft
model.add_adapter(config)



# Creating tokenizer and defining the pad token
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, padding_side='right')
tokenizer.pad_token = tokenizer.eos_token


# variant load model with AutoModelForCausalLM

In [19]:
!git clone https://github.com/Mike030668/MIPT_magistratura.git -q
!pip3 install pytelegrambotapi -q
!pip install trl transformers accelerate git+https://github.com/huggingface/peft.git -Uqqq
#!pip install bitsandbytes einops wandb -Uqqq

import gdown  # библиотека по работе с файлами в том числе и с гугл_диска
import pandas as pd
import numpy as np
import random
import torch
import gc
import sys

from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig

import warnings
warnings.filterwarnings('ignore')


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MAIN_DIR =  "/content/MIPT_magistratura/NLP_generation/Generation"

sys.path.append(MAIN_DIR)
from utils.download import gd_folder_download
from utils.constant import SHARE_DIR, TOKEN
from utils.generate import generate_answer, get_prompt, ROLES


# Подгружаем с публичной папки по https
dir_to = MAIN_DIR + "/TinyLamma/"

# код потребует гугл авторизацию любую из имеющихся
model_dir = gd_folder_download(SHARE_DIR, dir_to)

model = AutoModelForCausalLM.from_pretrained(model_dir)
model.to(DEVICE)

# Loading the model with double quantization
model_name =model.config._name_or_path

# Creating tokenizer and defining the pad token
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, padding_side='right')
tokenizer.pad_token = tokenizer.eos_token



# Loading the model with double quantization
model_name =model.config._name_or_path

# Creating tokenizer and defining the pad token
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, padding_side='right')
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

# Generation

In [133]:
def generate_answer(query, model, tokenizer, max_new_tokens, device, role):

    encoding = tokenizer(query, return_tensors="pt").to(device)
    generation_config = GenerationConfig(max_new_tokens=max_new_tokens,
                                         pad_token_id = tokenizer.eos_token_id,repetition_penalty=1.3,
                                         eos_token_id = tokenizer.eos_token_id)

    outputs = model.generate(input_ids=encoding.input_ids, generation_config=generation_config)
    text_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    ans = text_output.split(f"your role: {role}\n\n")[1].split("answer:")[1:]
    res = []
    for txt in ans:
       txt = txt.replace("[/INST]", " ").replace("\n", " ")
       if len(txt) > 5: res.append("".join(txt.split(".")[:2]))

    if len(res) == 1:
      return res[0]
    elif len(res)>1:
      return random.choice(res)

    else: return "I can't undanstand you, please answer me again"

def get_prompt(query, context, role = "Kyle"):
    prompt = f"<s>[INST]"
    prompt += f'Use the given context to guide your an about the query like indicated in your role'
    prompt += f"query: {query}\n\n"
    prompt += f"context: {context}\n\n"
    prompt += f"your role: {role}\n\n"
    prompt += f'answer:[/INST]</s>'

    return prompt



In [134]:
import telebot
#from telebot import types

# Создаем экземпляр бота
bot = telebot.TeleBot(TOKEN)

# Функция, обрабатывающая команду /start
@bot.message_handler(commands=["start"])
def start(m, res=False):
    bot.send_message(m.chat.id, 'Бот запущен. Начните общение')

conext_memory = ''
get_role = False
# Получение сообщений от пользователя
@bot.message_handler(content_types=["text"])
def handle_text(message):
    global conext_memory, get_role

    print("in_conext_memory", conext_memory)
    print()

    if not get_role:
        for role in ROLES:
          print("message.text", message.text)
          print(role)
          if role in message.text:
            print("get_role ", role)
            get_role = role
            break

        if not get_role: get_role = "Kyle"

    if get_role:
       prompt = get_prompt(query = message.text,
                           context = conext_memory,
                           role = get_role)
       print("have role" , get_role)


    gen_answer = generate_answer(prompt, model, tokenizer, role = get_role,
                                 max_new_tokens = 250, device = DEVICE)
    print("gen_answer", gen_answer)
    print()

    conext_memory = ''.join([conext_memory, message.text, gen_answer])

    print("out_conext_memory", conext_memory)
    # Отсылаем сообщение в чат пользователя
    try:
      bot.send_message(message.chat.id, gen_answer)
    except: pass
# Запускаем бота

bot.polling(none_stop=True, interval=0)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


in_conext_memory 

message.text What is your name ?
Stan
message.text What is your name ?
Kyle
message.text What is your name ?
Cartman
message.text What is your name ?
Randy
message.text What is your name ?
Butters
have role Kyle
gen_answer  !  I'm not gonna be a dick, dude

out_conext_memory What is your name ? !  I'm not gonna be a dick, dude


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


in_conext_memory What is your name ? !  I'm not gonna be a dick, dude

have role Kyle
gen_answer I don't know Do you? What do we do now? You can go home and get some sleep

out_conext_memory What is your name ? !  I'm not gonna be a dick, dudeWhat is your name ?I don't know Do you? What do we do now? You can go home and get some sleep


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


in_conext_memory What is your name ? !  I'm not gonna be a dick, dudeWhat is your name ?I don't know Do you? What do we do now? You can go home and get some sleep

have role Kyle
gen_answer Yeah!  That's right,  And then there was this one time when Kenny came upstairs from bed and said " Mom!" He didn't mean me at all, though  Then he started crying because of something he saw on TV last night

out_conext_memory What is your name ? !  I'm not gonna be a dick, dudeWhat is your name ?I don't know Do you? What do we do now? You can go home and get some sleepwhat is your name ?Yeah!  That's right,  And then there was this one time when Kenny came upstairs from bed and said " Mom!" He didn't mean me at all, though  Then he started crying because of something he saw on TV last night


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


in_conext_memory What is your name ? !  I'm not gonna be a dick, dudeWhat is your name ?I don't know Do you? What do we do now? You can go home and get some sleepwhat is your name ?Yeah!  That's right,  And then there was this one time when Kenny came upstairs from bed and said " Mom!" He didn't mean me at all, though  Then he started crying because of something he saw on TV last night

have role Kyle
gen_answer Dude!                                                             

out_conext_memory What is your name ? !  I'm not gonna be a dick, dudeWhat is your name ?I don't know Do you? What do we do now? You can go home and get some sleepwhat is your name ?Yeah!  That's right,  And then there was this one time when Kenny came upstairs from bed and said " Mom!" He didn't mean me at all, though  Then he started crying because of something he saw on TV last nightlet's speakDude!                                                             


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


in_conext_memory What is your name ? !  I'm not gonna be a dick, dudeWhat is your name ?I don't know Do you? What do we do now? You can go home and get some sleepwhat is your name ?Yeah!  That's right,  And then there was this one time when Kenny came upstairs from bed and said " Mom!" He didn't mean me at all, though  Then he started crying because of something he saw on TV last nightlet's speakDude!                                                             

have role Kyle
gen_answer Hey Hey

out_conext_memory What is your name ? !  I'm not gonna be a dick, dudeWhat is your name ?I don't know Do you? What do we do now? You can go home and get some sleepwhat is your name ?Yeah!  That's right,  And then there was this one time when Kenny came upstairs from bed and said " Mom!" He didn't mean me at all, though  Then he started crying because of something he saw on TV last nightlet's speakDude!                                                             are you robot?Hey Hey


In [2]:
!pip freeze > requirements.txt